## Prep

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

pd.options.display.max_rows = None

In [ ]:
mesa = pd.read_csv('../mesa/data_processed/Y_BaselineX_raw_full.csv')

jhs_outcome = pd.read_csv('../jhs/data/processed/jhs_cox_base.csv')
jhs_covar = pd.read_csv('../jhs/data/processed/jhs_raw_full.csv')

## Outcome

In [ ]:
mesa_outcome = mesa[['cvda','cvdatt']]

jhs_outcome = jhs_outcome[['event','time']]

In [ ]:
plt.hist(jhs_outcome['time'][jhs_outcome['event'] == 1], bins=10, alpha=0.5, label='JHS Event Time')
plt.hist(mesa_outcome['cvdatt'][mesa_outcome['cvda'] == 1], bins=10, alpha=0.5, label='MESA Event Time')

# Set axis labels and legend
plt.xlabel('Time')
plt.ylabel('Frequency of Events')
plt.legend()

# Show the plot
plt.title('Distribution of Event Times for JHS and MESA')
plt.show()

In [ ]:
sns.kdeplot(jhs_outcome['time'][jhs_outcome['event'] == 1], label='JHS', color='blue')
sns.kdeplot(mesa_outcome['cvdatt'][mesa_outcome['cvda'] == 1], label='MESA', color='red')

# Set axis labels and legend
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()

# Show the plot
plt.title('Density Plot of Event Time')
plt.show()


## nSES

In [ ]:
sns.kdeplot(jhs_covar['nbSESpc2score'], label='JHS', color='blue')
sns.kdeplot(mesa['F1_PC2'], label='MESA', color='red')

# Set axis labels and legend
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()

# Show the plot
plt.title('Density Plot of nSES')
plt.show()


## Covariates

In [ ]:
sns.kdeplot(jhs_covar['age'], label='JHS', color='blue')
sns.kdeplot(mesa['age'], label='MESA', color='red')

# Set axis labels and legend
plt.xlabel('Value')
plt.ylabel('Density')
plt.legend()

# Show the plot
plt.title('Density Plot of Baseline Age')
plt.show()


In [ ]:
mesa['race'].value_counts()